# MPC IP Driver for 4x40 QP Problem
MPC con ADMM. Las matrices de entrada constantes son globales y no hace falta ingresarlas. Se elige un número de iteraciones de ADMM, luego, cada iteración se ingresa x0 y se obtiene u0.
Interfaz axilite.


### Driver especializado

In [42]:
from pynq import DefaultIP
import struct

class mpcDriver(DefaultIP):
    
    # IP Register Map
    CTRL_REG = 0x0000
    GIE_REG = 0x0004
    INTE_REG = 0x0008
    INTS_REG = 0x000C
    
    x0_REG = 0x0010
    x0_0_REG = 0x0010
    x0_1_REG = 0x0018
    x0_2_REG = 0x0020
    x0_3_REG = 0x0028
    IT_REG = 0x0038
    u0_REG = 0x0030
    
    N_SYS = 4;
    N_QP = 4;
    M_QP = 40;
    
    def __init__(self, description):
        super().__init__(description=description)
        self.time_start = 0
        self.time_stop = 0

    bindto = ['xilinx.com:hls:mpc:1.0']

    def is_ready(self):
        state = self.read(mpcDriver.CTRL_REG)
        return (state >> 3) & 0x01

    def is_done(self):
        state = self.read(mpcDriver.CTRL_REG)
        return (state >> 1) & 0x01

    def is_idle(self):
        state = self.read(mpcDriver.CTRL_REG)
        return (state >> 2) & 0x01

    def start(self):
        self.write(mpcDriver.CTRL_REG, 0x00000001)

    def write_x0(self, data):
        if (len(data)!=self.N_SYS):
            print("Size error")
        else:
            # START TIMING
            #self.time_start = time.time_ns()
            #self.time_start = time.perf_counter_ns()
            offset = mpcDriver.x0_REG
            data_bytes = map(lambda x : struct.pack("f", x), data)
            for index, word_item in enumerate(data_bytes):
                self.write(offset, int.from_bytes(word_item,'little'))
                offset = offset + 8

    def write_IT(self, data):
        self.write(self.IT_REG, data)
    
    def read_u0(self):
        #STOP TIMING
        #self.time_stop = time.time_ns()
        #self.time_stop = time.perf_counter_ns()
        #if (self.read(mpcDriver.u0_CTRL_REG)):
        u0_0 = self.read(mpcDriver.u0_REG)
        u0_1 = self.read(mpcDriver.u0_REG+4)
        print(u0_0)
        print(u0_1)
        return (struct.unpack("f", u0_0.to_bytes(4,'little'))[0], struct.unpack("f", u0_1.to_bytes(4,'little'))[0])
        #return 1

    def get_extime(self):
        return self.time_stop-self.time_start

### Test

In [43]:
from pynq import Overlay
overlay = Overlay('/home/xilinx/pynq/overlays/der/mpc_eop_un02.bit')
mpc_ip = overlay.mpc_0

In [44]:
x0 = [2.0, 0.0, 40.0, 0.0]
mpc_ip.write_x0(x0)
mpc_ip.write_IT(10)

In [16]:
data_bytes = map(lambda x : struct.pack("f", x), [2.0, 0.0, 40.0, 0.0])
for index, word_item in enumerate(data_bytes):
    print(int.from_bytes(word_item,'little'))

1073741824
0
1109393408
0


In [38]:
mpc_ip.read(0x0038)

10

In [45]:
mpc_ip.start()
mpc_ip.read(0x0000)

14

In [40]:
mpc_ip.read(0x0034)

1022659576

In [46]:
mpc_ip.read_u0()

3210774955
1022659576


(-0.8771006464958191, 0.029851898550987244)

In [47]:
x0 = [-1.1628, 0.073162, 45.911, -2.8840]
mpc_ip.write_x0(x0)
mpc_ip.start()
mpc_ip.read(0x0000)

14

In [48]:
mpc_ip.read_u0()

1080442935
3199781748


(3.5976693630218506, -0.3609272241592407)

In [49]:
x0 = [-3.0766, 0.38776, 15.114, -1.9301]
mpc_ip.write_x0(x0)
mpc_ip.start()
mpc_ip.read(0x0000)

14

In [50]:
mpc_ip.read_u0()

1084187761
3209554051


(4.981010913848877, -0.8043290972709656)

In [44]:
mpc_ip.start()
mpc_ip.read(0x0000)

14

In [45]:
mpc_ip.read_u0()

3197259714


-0.28576475381851196

In [29]:
mpc_ip.read(0x0028)

0

In [31]:
d = mpc_ip.read(0x003c)
struct.unpack("f", d.to_bytes(4,'little'))[0]

0.0